In [1]:
#Import useful functions
import warnings
warnings.filterwarnings("ignore")
from sklearn import svm, datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix as cm
from sklearn.utils.multiclass import unique_labels
import itertools
from scipy import misc
import math
import os
import glob
import xlrd
import xlwt
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
from PIL import Image
from statsmodels.tsa.statespace.sarimax import SARIMAX
import statsmodels.api as sm
import pylab as plt
import numpy as np
from scipy.signal import argrelextrema
from IPython.display import Image
import matplotlib.patches as patches
import matplotlib as matplotlib
import xlsxwriter
from xlrd import open_workbook
from xlutils.copy import copy
from sklearn.model_selection import train_test_split
from datetime import datetime
plt.style.use('fivethirtyeight')
import itertools
from fbprophet import Prophet
from scipy.interpolate import interp1d
from sklearn.preprocessing import StandardScaler
import xlrd
import datetime

**ENTER THE MONTH AND YEAR CORRESPONDING TO THE DATA IN THE CELL BELOW**

In [2]:
monthyearbill='January 2020'

In [3]:
year=monthyearbill[-4:]
monthlist=['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
datelist=['-01-01','-02-01','-03-01','-04-01','-05-01','-06-01','-07-01', '-08-01','-09-01','-10-01','-11-01','-12-01']
yearlist=['2016', '2017', '2018', '2019', '2020', '2021']
yearind=yearlist==year
lastyearnum=yearlist[yearind-3]
month=monthyearbill[0:-5]
monthind=monthlist==month
billmonth=monthlist[monthind+1][0:3]
retmonth=monthlist[monthind-1][0:3]
monthyear=str(year)+str(datelist[monthind])
monthyear=pd.to_datetime(monthyear)
lastyear=pd.to_datetime(lastyearnum+datelist[monthind])
if monthind>3:
    lastQ=pd.to_datetime(str(year)+str(datelist[monthind-3]))
else:
    lastQ=pd.to_datetime(str(lastyearnum)+str(datelist[monthind+9]))

In [4]:
filename1 = '/Users/JoyThompson/Desktop/Contract Expiration Dates Auto Monthly.xlsx'
filename2 = '/Users/JoyThompson/Desktop/Client Billing Auto Monthly.xlsx'
filename3 = '/Users/JoyThompson/Desktop/Client Retention KPI Auto Monthly.xlsx'
billsheet='Billing Auto ' + billmonth
retsheet='Client Retention KPI Data ' + retmonth
calcsheet='Client Retention Calc ' + retmonth
contdates=pd.read_excel(filename1, sheetname=month)
billing=pd.read_excel(filename2, sheetname=billsheet)
alldata=pd.read_excel(filename3, sheetname=retsheet)
calcdata=pd.read_excel(filename3, sheetname=calcsheet)
poddata=pd.read_excel(filename3, sheetname='Pod vs Pod')
accdata=pd.read_excel(filename3, sheetname='Amazon Account Comparison')

In [5]:
newlist=[]
renewlist=[]
cancellist=[]
nonewlist=[]
for ind in contdates.index:
    isnew=contdates.loc[ind, 'Client New or Renew']
    client=contdates.loc[ind, 'Client Name']
    if isnew == 'New':
        newlist.append(client)
    if isnew == 'Renew':
        renewlist.append(client)
    if isnew == 'Cancel':
        cancellist.append(client)
    if isnew == 'No Renew':
        nonewlist.append(client)
contdates=contdates.set_index('Client Name')
renewrate=len(renewlist)/(len(renewlist) +len(nonewlist))
optrate=len(cancellist)/len(contdates)

In [6]:
def vsanalysis(podacname, podacdata, podorac, monthyear, lastyear):
    lencli=len(contdates[contdates[podorac]==podacname])
    podsdata=contdates[contdates[podorac]==podacname]
    podsdata2=podacdata[podacdata[podorac]==podacname]
    old=0
    for data in podsdata.index:
        start=podsdata.loc[data, 'Initial Start Date']
        if start>lastyear:
            old=old+1

    lifeav=np.mean(contdates.loc[contdates[podorac]==podacname, "Age of Client Relationship (Years)"])
    kpilist=[renewlist, nonewlist, cancellist]
    renew=0
    can=0
    nonew=0
    count=0
    kpiout=[renew, nonew, can]
    def kpifunc(kpilist, podacname):
        kpiout=0
        for kpil in kpilist:
            podac=contdates.loc[kpil, podorac]
            if podac ==podacname:
                kpiout=kpiout+1
        return kpiout
    renew=kpifunc(renewlist, podacname)
    nonew=kpifunc(nonewlist, podacname)
    can=kpifunc(cancellist, podacname)
    noretain=nonew
    for kpi in nonewlist:
        start=podsdata.loc[data, 'Initial Start Date']
        if start>lastyear:
            noretain=noretain+1
            
        
    if float(renew+nonew)!=0:
        rerate=renew/(renew+nonew)
    else:
        rerate=0
    opt=can/lencli
    print(opt)
    lostsum=sum(podsdata2['Number Unretained'])+noretain
    reten=np.round(float((old-lostsum)/old), decimals=2)
    podacdata=podacdata.append({podorac: podacname, 'Month': monthyear, 'Client Opt Out %': opt, 'Client Renewal Rate': rerate, 'Average Client Lifetime': lifeav,
                               'Client Retention Rate':reten, 'Number Cancelled': can, 'Number Renewed':renew,
                               "Number Unrenewed": nonew, 'Number Unretained': noretain}, ignore_index=True)
    return podacdata

In [7]:
poddata=vsanalysis("Brian's Pod", poddata, 'Pod', monthyear, lastyear)
poddata=vsanalysis("Spencer's Pod", poddata, 'Pod', monthyear, lastyear)
poddata=vsanalysis("On - Boarding", poddata, 'Pod', monthyear, lastyear)
accdata=vsanalysis("VC", accdata, 'VC, SC, or Hybrid', monthyear, lastyear)
accdata=vsanalysis("SC", accdata, 'VC, SC, or Hybrid', monthyear, lastyear)
accdata=vsanalysis("Hybrid", accdata, 'VC, SC, or Hybrid', monthyear, lastyear)
print(poddata)

0.05
0.05555555555555555
0.0
0.0
0.05714285714285714
0.0
             Pod      Month  Client Opt Out %  Client Renewal Rate  \
0  Spencer's Pod 2019-12-01          0.086957             0.333333   
1    Brian's Pod 2019-12-01          0.047619             0.750000   
2  On - Boarding 2019-12-01          0.000000             1.000000   
3    Brian's Pod 2020-01-01          0.050000             0.750000   
4  Spencer's Pod 2020-01-01          0.055556             0.700000   
5  On - Boarding 2020-01-01          0.000000             1.000000   
6    Brian's Pod 2020-01-01          0.050000             0.333333   
7  Spencer's Pod 2020-01-01          0.055556             1.000000   
8  On - Boarding 2020-01-01          0.000000             0.000000   

   Average Client Lifetime  Client Retention Rate  Number Cancelled  \
0                 1.000000               0.666667                 2   
1                 1.000000               0.857143                 1   
2                 0.249315   

In [8]:
for client in newlist:
    start=contdates.loc[client, 'Initial Start Date']
    end=contdates.loc[client, 'Contract End Date']
    alldata=alldata.append({'Account': client, 'Contract Start Date': start, 'Contract End Date': end}, ignore_index=True)
alldata=alldata.set_index('Account')
for  client in nonewlist:
    cancel=contdates.loc[client, 'Client Lifetime End Date' ]
    alldata.loc[client, 'Cancel Date']=cancel

for  client in cancellist:
    cancel=contdates.loc[client, 'Client Lifetime End Date' ]
    alldata.loc[client, 'Cancel Date']=cancel
    
for client in renewlist:
    end=contdates.loc[client, 'Contract End Date']
    alldata.loc[client, 'Contract End Date']=end

In [9]:
billing=billing.set_index('Client Name')
alldata[monthyear]=' '
for ind in billing.index:
    bill=billing.loc[ind, monthyearbill]
    alldata.loc[ind, monthyear]=bill
active=len(billing)
avrev=np.round(np.mean(billing.loc[:, monthyearbill]), decimals=2)
sumrev=np.round(np.sum(billing.loc[:, monthyearbill]), decimals=2)

In [10]:
retain=0
retainQ=0
revret=0

for ind in alldata.index: 
    cancel=alldata.loc[ind, 'Cancel Date']
    start=alldata.loc[ind, 'Contract Start Date']
    end=alldata.loc[ind, 'Contract End Date']
    revenue=alldata.loc[ind, monthyear]
    if str(cancel)[0] != 'N':
        lifetime=cancel-start
        lifetime=str(lifetime)[0:3]
        lifetime=np.round(float(lifetime)/365, decimals=2)
        alldata.loc[ind, 'Client Lifetime (Years)']=lifetime
    else:
        lifetime=end-start
        lifetime=str(lifetime)[0:4]
        lifetime=np.round(float(lifetime)/365, decimals=2)
        alldata.loc[ind, 'Client Lifetime (Years)']=lifetime
        if start <= lastyear:
            retain=retain+1
            revret=revret+revenue
        if start <=lastQ:
            retainQ=retainQ+1

In [11]:
lifetimeav=np.round(np.mean(alldata.loc[:, 'Client Lifetime (Years)']), decimals=2)
calcdata=calcdata.set_index('CLIENT RETENTION KPIS')
clientslast=calcdata.loc['TOTAL CLIENTS', lastyear]
clientsQ=calcdata.loc['TOTAL CLIENTS', lastQ]
revlast=calcdata.loc['TOTAL MONTHLY REVENUE', lastyear]
retainper=retain/clientslast
retainperQ=retainQ/clientsQ
revper=revret/revlast
churn= (1/(1-retainperQ))/4
avltval=avrev*churn*12
calcdata.loc['AVERAGE MONTHLY CLIENT REVENUE',monthyear]=avrev
calcdata.loc['TOTAL MONTHLY REVENUE',monthyear]=sumrev
calcdata.loc['TOTAL CLIENTS',monthyear]=active
calcdata.loc['TOTAL CLIENTS RETAINED ANNUALLY',monthyear]=retain
calcdata.loc['TOTAL CLIENTS LAST YEAR',monthyear]=clientslast
calcdata.loc['CLIENT RETENTION RATE ANNUAL',monthyear]=retainper
calcdata.loc['TOTAL CLIENTS RETAINED QUARTERLY',monthyear]=retainQ
calcdata.loc['TOTAL CLIENTS LAST QUARTER',monthyear]=clientsQ
calcdata.loc['CLIENT RETENTION RATE QUARTER',monthyear]=retainperQ
calcdata.loc['REVENUE RETAINED ANNUALLY',monthyear]=revret
calcdata.loc['REVENUE LAST YEAR',monthyear]=revlast
calcdata.loc['RETAINED REVENUE RATE ANNUAL',monthyear]=revper
calcdata.loc['CLIENT LIFETIME (CHURN)',monthyear]=churn
calcdata.loc['CLIENT LIFETIME (YEARS)',monthyear]=lifetimeav
calcdata.loc['CLIENT RENEWAL RATE',monthyear]=renewrate
calcdata.loc['CLIENT OPT OUT RATE',monthyear]=optrate
calcdata.loc['AVERAGE CUSTOMER LIFETIME VALUE',monthyear]=avltval

In [12]:
filenamefinal= '/Users/JoyThompson/Desktop/RetentionKPIAutomated2' + billmonth +  '.xlsx'
writer = pd.ExcelWriter(filenamefinal, engine='xlsxwriter')
alldata.to_excel(writer, sheet_name='Retention KPI')
calcdata.to_excel(writer, sheet_name='Retention KPI Calc')
accdata.to_excel(writer, sheet_name='Account Data')
poddata.to_excel(writer, sheet_name='Pod Data')
writer.save()